## Loading Libraries

In [2]:
import pandas as pd
import numpy as np
import joblib

## ML libraries
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from skopt import BayesSearchCV
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score

## visalization libraries
import seaborn as sns
import plotly.express as px

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Patch

pd.options.display.max_columns = None
pd.set_option('display.float_format', '{:2f}'.format)

## Zone Info

In [3]:
## Strike zone
## assuming view is from pitchers mound

lower_boundary = 1.5
upper_boundary = 3.5
left_boundary = -0.833
right_boundary = 0.833

heart_lower_boundary = 1.833
heart_upper_boundary = 3.167
heart_left_boundary = -0.558
heart_right_boundary = 0.558

shadow_lower_boundary = 1.167
shadow_upper_boundary = 3.833
shadow_left_boundary = -1.108
shadow_right_boundary = 1.108

chase_lower_boundary = 0.5
chase_upper_boundary = 4.333
chase_left_boundary = -1.667
chase_right_boundary = 1.667

In [4]:
master_trackman = pd.read_csv('../../Trackman CSV Files/merged_trackman_games.csv')
master_trackman.head()

/var/folders/bp/l1vb_64s6tj40wftrxyjhnfm0000gn/T/ipykernel_15546/2981408051.py:1: DtypeWarning: Columns (91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  master_trackman = pd.read_csv('../../Trackman CSV Files/merged_trackman_games.csv')


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",831228.000000,Right,WAS_HUS,"Ishikawa, Keni",1000274623.000000,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.028060,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.621390,0.510940,5.836200,-16.090760,18.231070,4.190400,1.212030,-0.291350,83.110150,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.463980,10.486150,-0.412500,50.000000,5.342850,2.649920,-130.679410,-8.015750,-4.299720,25.181650,-13.875380,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.391770,NaN,NaN,6.917910,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.000000,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1000114026.000000,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.662890,-131.574870,12.590830,5.619120,-7.522340,-6.937690,-0.509450,2.802820,-2.149860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",831228.000000,Right,WAS_HUS,"Ishikawa, Keni",1000274623.000000,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.167840,1.675543,-0.415000,2782.722426,155.127014,11:15,5.533740,0.741550,4.993150,-35.983520,6.645160,-2.487760,4.117590,0.142400,75.363000,-4.887176,-0.853995,0.469918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.118030,3.249250,-0.699510,50.000000,5.665960,0.971940,-119.219350,2.168950

## Data Manipulation

In [6]:
def separate_by_zone(df):
    zone_df = df.copy()
    zone_df['PlateLocSide'] *= -1
    
    # Default all rows to Waste
    zone_df['PlateZone'] = 'Waste'

    # Heart zone
    mask_heart = (
        (zone_df['PlateLocSide'] >= heart_left_boundary) &
        (zone_df['PlateLocSide'] <= heart_right_boundary) &
        (zone_df['PlateLocHeight'] >= heart_lower_boundary) &
        (zone_df['PlateLocHeight'] <= heart_upper_boundary)
    )
    zone_df.loc[mask_heart, 'PlateZone'] = 'Heart'

    # Shadow zone
    mask_shadow = (
        (zone_df['PlateLocSide'] >= shadow_left_boundary) &
        (zone_df['PlateLocSide'] <= shadow_right_boundary) &
        (zone_df['PlateLocHeight'] >= shadow_lower_boundary) &
        (zone_df['PlateLocHeight'] <= shadow_upper_boundary)
    )
    zone_df.loc[mask_shadow & ~mask_heart, 'PlateZone'] = 'Shadow'

    # Chase zone
    mask_chase = (
        (zone_df['PlateLocSide'] >= chase_left_boundary) &
        (zone_df['PlateLocSide'] <= chase_right_boundary) &
        (zone_df['PlateLocHeight'] >= chase_lower_boundary) &
        (zone_df['PlateLocHeight'] <= chase_upper_boundary)
    )
    zone_df.loc[mask_chase & ~mask_heart & ~mask_shadow, 'PlateZone'] = 'Chase'

    return zone_df

zone_separated = separate_by_zone(master_trackman)
zone_separated.sample(12)
 

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,PlateZone
591484,21,2025-03-15,16:17:26.98,2,2,"Jones, Merit",1000196598.000000,Right,UTA_UTE,"Inoue, Shintaro",830359.000000,Left,KAN_WIL,Undefined,1,Bottom,1,0,1,Fastball,Four-Seam,FoulBallNotFieldable,Undefined,Undefined,Undefined,0,0,91.825760,-1.739459,-3.006936,2166.576943,206.251812,1:00,6.123430,2.195580,6.292780,-19.024930,13.700640,6.239730,2.932380,0.057920,83.901330,-5.344829,-1.880552,0.411732,90.184110,30.681840,-30.546855,4222.064625,86.526100,60.982920,-67.920740,289.028640,175.792720,-47.916963,4.635080,-4.095360,7.932960,-1.977040,50.000000,5.985940,6.744270,-133.258600,-4.619290,-7.354110,28.999590,-17.928700,KAN_WIL,UTA_UTE,KansasState,D1,BIG12,20250315-KansasState-1,e5b47150-01e2-11f0-a380-3b2467bf6551,91.545970,72.966740,NaN,7.924430,NaN,NaN,NaN,-0.241190,4.801040,-2.279640,3260fa9d-8b32-4332-a400-368e53921f8a,2025-03-15,21:17:26.98,2025-03-15T16:17:26.9874176-05:00,2025-03-15T21:17:26.9874176Z,FlyBall,v3,471328.000000,471475.000000,15 March 2025 17:00-utah@kansasst,"Kilgore, Cal",808771.000000,Right,UTA_UTE,c3ae716a-ff8c-4256-946b-de216fe538ec,54.207500,-134.171110,14.499790,6.122420,-4.055140,-8.964350,-2.192900,6.975670,-3.677060,140.154092,-0.239200,97.926020,-28.960560,7.871610,-1.816840,0.370050,-0.069830,0.009720,-0.000610,4.802680,67.450360,-17.877540,-0.068980,0.775970,-0.388470,0.106330,-0.014770,0.000800,-2.280740,-57.791770,2.027290,1.289410,-0.746470,0.240960,-0.048410,0.005270,-0.000230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,Medium,Low,NaN,NaN,NaN,Heart
50187,16,2025-04-06,11:12:20.29,5,1,"Sutton, Carter",690159.000000,Right,LAM_CAR,"Melendez, Xavier",10159408.000000,Right,ARL_MAV,Undefined,1,Top,2,0,0,Sinker,Sinker,InPlay,Undefined,LineDrive,Out,1,0,89.494490,-0.36